# Load rankings and categories

In [1]:
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import json,os,pdb
import tqdm
import warnings
import yfinance as yf

# Load data


In [2]:
df = pd.read_parquet('~/work/project/daily_20221004.parquet.gzip') # modify the path according to your needs
df.columns = [c.split('.')[-1].replace('_','') for c in df.columns]
df.index = df.index.set_levels([df.index.levels[0], pd.to_datetime(df.index.levels[1])])

Load the listing file and check for the rankings

In [3]:
with open('listing_map_20221004.json',encoding='utf-8') as json_data:
    data = json.load(json_data)
listmap = pd.DataFrame(data['data'])
listmap.sort_values('cmc_rank',inplace=True)
listmap.set_index('cmc_rank',inplace=True)
listmap['symbol_id'] = listmap['symbol']+ '_'+ listmap['id'].astype(str)

In [4]:
df.head()

timeopen  \
symbol_id time_close                                                   
MPH_7742  2020-11-23 00:59:59.999000+00:00  2020-11-23T00:00:00.000Z   
          2020-11-24 00:59:59.999000+00:00  2020-11-24T00:00:00.000Z   
          2020-11-25 00:59:59.999000+00:00  2020-11-25T00:00:00.000Z   
          2020-11-26 00:59:59.999000+00:00  2020-11-26T00:00:00.000Z   
          2020-11-27 00:59:59.999000+00:00  2020-11-27T00:00:00.000Z   

                                                            timehigh  \
symbol_id time_close                                                   
MPH_7742  2020-11-23 00:59:59.999000+00:00  2020-11-23T00:00:07.000Z   
          2020-11-24 00:59:59.999000+00:00  2020-11-24T00:00:07.000Z   
          2020-11-25 00:59:59.999000+00:00  2020-11-25T00:00:06.000Z   
          2020-11-26 00:59:59.999000+00:00  2020-11-26T00:56:05.000Z   
          2020-11-27 00:59:59.999000+00:00  2020-11-27T00:57:05.000Z   

                                                             timelow  \
symbol_id time_close                                                   
MPH_7742  2020-11-23 00:59:59.999000+00:00  2020-11-23T00:37:05.000Z   
          2020-11-24 00:59:59.999000+00:00  2020-11-24T00:40:06.000Z   
          2020-11-25 00:59:59.999000+00:00  2020-11-25T00:58:05.000Z   
          2020-11-26 00:59:59.999000+00:00  2020-11-26T00:13:06.000Z   
          2020-11-27 00:59:59.999000+00:00  2020-11-27T00:16:05.000Z   

                                                 open       high        low  \
symbol_id time_close                                                          
MPH_7742  2020-11-23 00:59:59.999000+00:00  90.554933  90.554933  79.852330   
          2020-11-24 00:59:59.999000+00:00  85.161030  85.161030  80.694133   
          2020-11-25 00:59:59.999000+00:00  57.527366  57.527366  53.679986   
          2020-11-26 00:59:59.999000+00:00  39.466839  40.121814  38.980499   
          2020-11-27 00:59:59.999000+00:00  30.002092  30.391962  29.690305   

                                                close        volume  \
symbol_id time_close                                                  
MPH_7742  2020-11-23 00:59:59.999000+00:00  85.599205  1.600851e+07   
          2020-11-24 00:59:59.999000+00:00  81.645754  1.009738e+07   
          2020-11-25 00:59:59.999000+00:00  53.704963  8.899399e+06   
          2020-11-26 00:59:59.999000+00:00  40.056399  4.225065e+06   
          2020-11-27 00:59:59.999000+00:00  30.383005  2.933446e+06   

                                            marketcap  \
symbol_id time_close                                    
MPH_7742  2020-11-23 00:59:59.999000+00:00        0.0   
          2020-11-24 00:59:59.999000+00:00        0.0   
          2020-11-25 00:59:59.999000+00:00        0.0   
          2020-11-26 00:59:59.999000+00:00        0.0   
          2020-11-27 00:59:59.999000+00:00        0.0   

                                                           timestamp symbol  \
symbol_id time_close                                                          
MPH_7742  2020-11-23 00:59:59.999000+00:00  2020-11-23T00:59:59.999Z    MPH   
          2020-11-24 00:59:59.999000+00:00  2020-11-24T00:59:59.999Z    MPH   
          2020-11-25 00:59:59.999000+00:00  2020-11-25T00:59:59.999Z    MPH   
          2020-11-26 00:59:59.999000+00:00  2020-11-26T00:59:59.999Z    MPH   
          2020-11-27 00:59:59.999000+00:00  2020-11-27T00:59:59.999Z    MPH   

                                              id  
symbol_id time_close                              
MPH_7742  2020-11-23 00:59:59.999000+00:00  7742  
          2020-11-24 00:59:59.999000+00:00  7742  
          2020-11-25 00:59:59.999000+00:00  7742  
          2020-11-26 00:59:59.999000+00:00  7742  
          2020-11-27 00:59:59.999000+00:00  7742

# Begining Analysis

In [5]:
df2 = df.copy() # makes copy of original df
df2.reset_index(inplace = True)
df2['year'] = pd.DatetimeIndex(df2['timeopen']).year # adds year column to data frame
df2['month'] = pd.DatetimeIndex(df2['timeopen']).month  # adds month column to data frame
df2['day'] = pd.DatetimeIndex(df2['timeopen']).day  # adds day column to data frame
df2_first = df2.drop_duplicates(subset='symbol_id') #finds the first entry for each symbol
df2_last = df2.drop_duplicates(subset='symbol_id', keep='last').copy() #finds the last symbol for each symbol
df2_last['market_perc'] = df2_last['marketcap']/ df2_last['marketcap'].sum() # finds the market cap percentage. Used last known marketcap to calc
df2_last = df2_last.sort_values('market_perc', ascending=False) # sort from highest market cap percent to lowest
df2_last['cum_perc'] = df2_last['market_perc'].cumsum() # creates a column for the cumulative percentage

In [6]:
df2_last.head(10) #shows the top 10 highest market cap coins

,symbol_id,time_close,timeopen,timehigh,timelow,open,high,low,close,volume,marketcap,timestamp,symbol,id,year,month,day,market_perc,cum_perc
1010930,BTC_1,2022-10-04 00:59:59.999000+00:00,2022-10-04T00:00:00.000Z,2022-10-04T00:20:00.000Z,2022-10-04T00:59:00.000Z,19623.584689,19682.097005,19569.057599,19569.057599,3.019521e+10,3.751072e+11,2022-10-04T00:59:59.999Z,BTC,1,2022,10,4,0.359519,0.359519
1140492,ETH_1027,2022-10-04 00:59:59.999000+00:00,2022-10-04T00:00:00.000Z,2022-10-04T00:17:00.000Z,2022-10-04T00:59:00.000Z,1323.278296,1327.501947,1321.478301,1321.478301,9.889020e+09,1.620678e+11,2022-10-04T00:59:59.999Z,ETH,1027,2022,10,4,0.155333,0.514852
1217597,USDT_825,2022-10-04 00:59:59.999000+00:00,2022-10-04T00:00:00.000Z,2022-10-04T00:39:00.000Z,2022-10-04T00:10:00.000Z,1.000091,1.000136,1.000065,1.000099,3.948120e+10,6.795617e+10,2022-10-04T00:59:59.999Z,USDT,825,2022,10,4,0.065132,0.579984
224969,USDC_3408,2022-10-04 00:59:59.999000+00:00,2022-10-04T00:00:00.000Z,2022-10-04T00:21:00.000Z,2022-10-04T00:05:00.000Z,0.999916,1.000129,0.999848,0.999932,4.347674e+09,4.717649e+10,2022-10-04T00:59:59.999Z,USDC,3408,2022,10,4,0.045216,0.625200
591885,BNB_1839,2022-10-04 00:59:59.999000+00:00,2022-10-04T00:00:00.000Z,2022-10-04T00:17:00.000Z,2022-10-04T00:59:00.000Z,287.024182,287.688765,286.528964,286.528964,6.934432e+08,4.622780e+10,2022-10-04T00:59:59.999Z,BNB,1839,2022,10,4,0.044307,0.669506
57598,LUNA_4172,2022-04-18 00:59:59.999000+00:00,2022-04-18T00:00:00.000Z,2022-04-18T00:59:00.000Z,2022-04-18T00:10:00.000Z,77.445059,78.015995,77.039673,78.015995,1.134889e+09,2.781884e+10,2022-04-18T00:59:59.999Z,LUNA,4172,2022,4,18,0.026663,0.696169
664711,XRP_52,2022-10-04 00:59:59.999000+00:00,2022-10-04T00:00:00.000Z,2022-10-04T00:00:00.000Z,2022-10-04T00:58:00.000Z,0.462878,0.462878,0.456886,0.456954,1.828199e+09,2.280231e+10,2022-10-04T00:59:59.999Z,XRP,52,2022,10,4,0.021855,0.718024
935969,BUSD_4687,2022-10-04 00:59:59.999000+00:00,2022-10-04T00:00:00.000Z,2022-10-04T00:15:00.000Z,2022-10-04T00:40:00.000Z,1.000200,1.000815,0.999495,1.000401,6.356159e+09,2.105067e+10,2022-10-04T00:59:59.999Z,BUSD,4687,2022,10,4,0.020176,0.738200
831362,UST_7129,2022-04-18 00:59:59.999000+00:00,2022-04-18T00:00:00.000Z,2022-04-18T00:56:00.000Z,2022-04-18T00:03:00.000Z,1.001566,1.001987,1.001280,1.001743,2.391945e+08,1.747529e+10,2022-04-18T00:59:59.999Z,UST,7129,2022,4,18,0.016749,0.754949
350442,ADA_2010,2022-10-04 00:59:59.999000+00:00,2022-10-04T00:00:00.000Z,2022-10-04T00:17:00.000Z,2022-10-04T00:59:00.000Z,0.428003,0.428976,0.426254,0.426254,4.408164e+08,1.460054e+10,2022-10-04T00:59:59.999Z,ADA,2010,2022,10,4,0.013994,0.768942


In [7]:
df_cutoff = df2_last.loc[df2_last['cum_perc'] <= .91]
tethers = df_cutoff.loc[(df_cutoff['close'] > .99) & (df_cutoff['close'] < 1.01)]['symbol_id'].values # removes 4 tether coins
df_cutoff = df_cutoff.loc[~df_cutoff['symbol_id'].isin(tethers)]
df2_cutoff = df2_first.loc[df2_first['symbol_id'].isin(df_cutoff['symbol_id'])]
symbols = df2_cutoff.loc[df2_cutoff['year'] > 2020]['symbol_id'].values
symbols2 = df2_cutoff.loc[(df2_cutoff['year'] == 2020) & (df2_cutoff['month'] > 8)]['symbol_id'].values
all_symbols = np.concatenate((symbols, symbols2))

In [8]:
print(len(df2_cutoff))

40


In [9]:
print(len(all_symbols))

11


In [10]:
df3 = df.copy()
df3.reset_index(inplace = True)
df3 = df3.loc[df3['symbol_id'].isin(df_cutoff['symbol_id'])]
df3 = df3.loc[~df3['symbol_id'].isin(all_symbols)]

In [11]:
df3['time_close'] = df3['time_close'].dt.date
df3 = df3[(df3['time_close'] >= df3.loc[df3['symbol_id'] == 'DOT_6636']['time_close'].min())]
df3.set_index(['time_close'],inplace = True)

In [12]:
df3.head()

,symbol_id,timeopen,timehigh,timelow,open,high,low,close,volume,marketcap,timestamp,symbol,id
time_close,,,,,,,,,,,,,
2020-08-21,LUNA_4172,2020-08-21T00:00:00.000Z,2020-08-21T00:15:10.000Z,2020-08-21T00:30:16.000Z,0.514707,0.516277,0.514387,0.514751,NaN,1.984236e+08,2020-08-21T00:59:59.999Z,LUNA,4172
2020-08-22,LUNA_4172,2020-08-22T00:00:00.000Z,2020-08-22T00:11:04.000Z,2020-08-22T00:58:14.000Z,0.433162,0.434034,0.421722,0.421722,NaN,1.625672e+08,2020-08-22T00:59:59.999Z,LUNA,4172
2020-08-23,LUNA_4172,2020-08-23T00:00:00.000Z,2020-08-23T00:01:22.000Z,2020-08-23T00:32:14.000Z,0.468375,0.468722,0.463915,0.464550,NaN,1.790803e+08,2020-08-23T00:59:59.999Z,LUNA,4172
2020-08-24,LUNA_4172,2020-08-24T00:00:00.000Z,2020-08-24T00:09:22.000Z,2020-08-24T00:34:08.000Z,0.483165,0.483645,0.481652,0.482459,NaN,1.859923e+08,2020-08-24T00:59:59.999Z,LUNA,4172
2020-08-25,LUNA_4172,2020-08-25T00:00:00.000Z,2020-08-25T00:04:13.000Z,2020-08-25T00:51:08.000Z,0.529584,0.529586,0.517749,0.519632,NaN,2.003271e+08,2020-08-25T00:59:59.999Z,LUNA,4172


In [13]:
TICKER = 'SPY'
START_DATE = '2020-08-21'
END_DATE = '2022-10-4'
spy = yf.download(TICKER, 
                 start=START_DATE, 
                 end=END_DATE,
                 progress=False)

In [14]:
TICKER = '^VIX'
START_DATE = '2020-08-21'
END_DATE = '2022-10-4'
vix = yf.download(TICKER, 
                 start=START_DATE, 
                 end=END_DATE,
                 progress=False)

In [15]:
df3['spy'] = spy['Adj Close']

In [16]:
df3['vix'] = vix['Adj Close']

In [17]:
df3['spy'] = df3.groupby(['symbol_id'])['spy'].ffill()

In [18]:
df3['vix'] = df3.groupby(['symbol_id'])['vix'].ffill()

In [21]:
df3.drop(columns=['timeopen', 'timehigh', 'timelow', 'open', 'high', 'low', 'timestamp', 'symbol', 'id'], inplace = True)
df3.reset_index(inplace = True)
df3.set_index(['symbol_id', 'time_close'],inplace = True)

In [28]:
df3.head()

close  volume     marketcap         spy        vix
symbol_id time_close                                                       
LUNA_4172 2020-08-21  0.514751     NaN  1.984236e+08  328.459320  22.540001
          2020-08-22  0.421722     NaN  1.625672e+08  328.459320  22.540001
          2020-08-23  0.464550     NaN  1.790803e+08  328.459320  22.540001
          2020-08-24  0.482459     NaN  1.859923e+08  331.787720  22.370001
          2020-08-25  0.519632     NaN  2.003271e+08  332.948761  22.030001

In [29]:
df3.to_csv('clean_df.csv',index=True)